##### 目前流程借鑑資料採礦流程CRISP，優點可以分割成不同子任務，方便執行
###### 圖片來源：https://www.slideshare.net/brian401777/ss-34673079

<img style="float: left;" src="https://image.slidesharecdn.com/random-140514083724-phpapp02/95/-11-1024.jpg?cb=1400057064,width=10" width="45%">

<img style="float: right;" src="https://image.slidesharecdn.com/random-140514083724-phpapp02/95/-12-1024.jpg?cb=1400057064" width="45%">

## 定義問題

##### 歸納問題與定義建模
* 定義建模：選擇**預測性建模**(評估未知值+邏輯敘述 => 選擇**分類模型**or**迴歸模型**)
* 目標：本資料集針對**客戶流失傾向**進行討論
* 模型選擇：如果以單純流失傾向，可用**分類模型**為主(較少受到其他統計處理的關注)，模型採用(機率的)**演繹**程序。

##### 建立監督式區隔模型
* 建立可理解的監督式區隔，以例如「年齡30~35歲月租自動續約的男性平均顧客流失率5%」做區隔表達。
* 找出不同區隔資料中具有**顯著差異的欄位**

## 定義分析資料

##### 需思考的問題：
* 解決**數據不平衡**的問題(is churn與not churn比例相差懸殊)，數據不平衡會造成訓練的模型準確度極高，但拿來測試就不準的問題
* 如何建立連續型數值監督式區隔的情況

## 數據理解

* 欄位主要以客戶個人資料(所處城市、年齡、性別等)、交易資料(方案價格、方案天數等)、用戶紀錄(聽歌長度比例、聽歌秒數，以每一天為一筆資料)

## 資料準備(預處理)

* 處理缺失值 => 目前觀察其他人的做法，不建議直接將缺失值移除，而是賦予值**歸於同一類**
* 處理各欄位不合理的值
* 處理各欄位的型態(例如經過one-hot encoding的資料雖然看起來是數字，但型態要轉成字串等等)
* Python型態問題須考慮記憶體問題，例如float64較float32占較多空間；或是日期型態以datetime型態存在可以處理日期的分隔與取用

## 資料準備(特徵工程)

* 各欄位的意義，定義問題(影響欄位選擇與整併)
* 欄位選擇決定(差異太大的，如city)(套入weka先行確認?)
* 各欄位整併(移除不合理的資料 or 分割連續資料為離散(做類別) or 例如將日期分類成不同月份、季節、周末與平日)

## 模型建立&預測&評估

##### 關於模型的思考
* 找出可用的模型(分類或迴歸或其他)
* 選擇模型的優勢與劣勢(例如隨機森林能處理特徵很多的資料，且訓練完會給出哪些特徵很重要)
* 模型怎麼設定參數(大部分應該都有部分內建的參數設定；但調參應該不是最優先要做的事，不過在測試過程中確認不同參數或算法影響模型訓練結果程度)
* 訓練結果的驗證效果(驗證模型的分數或其他驗證方法)
* 評估測試結果的方法(例如FPR,TPR等)

## 模型選擇

> ##### 決策樹
> * 商業應用上，一般會做**分類機率評估**，而非單純的分類
> * 特徵對於熵值的**資訊增益(Information Gain)**越大，分類的效果(重要性)越好
> * 資訊增益(IG)解決了資料區隔純度低的問題，並賦予各欄位特徵不同權重
> * **連續型資料**透過所有合理的分隔方式來進行**資料增益的比較**，可以選擇出合理的分隔方式
> * 利用機率評估樹得到各種特徵選擇順序，由於重要順序是相互牽連的，決定拿掉重要性低的特徵後，剩下特徵仍需要進行模型再度確認各特徵的重要順序

> ##### 邏輯迴歸
> 

> ##### 支援向量機(SVM)
> * 提供落在分類邊界錯誤側的資料點予以懲罰 => loss function
> * loss function最常使用的就是**平方誤差**(優點：方便；缺點：也會懲罰到正確側的資料點)

## 部屬
##### 將發現的結果與過程組織形成結果報告

## 後續應用

##### 待討論...

---

## 問題點

##### 數據不平衡問題
> * 分類問題中，某分類樣本數量遠大於另一分類樣本數量
> * 缺點：可能會產生一個局部優化的分類模型
> * 不平衡數據衡量標準：
>> * 無法使用準確率進行衡量模型
>> * 可以使用混淆矩陣(TP、TN、FP、FN)以及ROC(含AUC)進行衡量模型
> * 處理數據不平衡的幾種常見方法
>> * 降採樣：刪除大類別樣本來取得與小樣本的平衡(缺點：拋棄潛在有用的數據)
>> * 過採樣(單純重複小樣本)：缺點：增加overfitting的風險
>> * 過採樣(SMOTE)：通過在小類別樣本之間插值的方法產生新的小類別樣本，來對訓練集進行過采樣(缺點：可能生成可疑的點)
>> <img style="float: center;" src="http://www.aprilzephyr.com/images/Imbalance/2.png" width="60%">


參考網站：http://www.aprilzephyr.com/blog/07302016/%E6%B7%B1%E5%85%A5%E7%90%86%E8%A7%A3%E4%B8%8D%E5%B9%B3%E8%A1%A1%E6%95%B8%E6%93%9A%E5%88%86%E9%A1%9E%E5%95%8F%E9%A1%8C-%E8%BD%89/
https://www.zhihu.com/question/269698662

##### Transaction與Train+Members合併問題 
* 因為Transaction與Train+Members欄位不相同(msno重複與不重複)，所以須解決如何進行合併比較合理